In [4]:
from models.llm_dataset import LLMType, LLMDataset, LLM1DatasetRow, LLM2DatasetRow, DatasetRow


dataset = LLMDataset.from_file()

In [ ]:
dataset.get_llm_type_rows(LLMType.LLM1).fill_systems(["a", "b", "c"]).rows

In [55]:
from datasets import Dataset

# Dataset.from_list([x.model_dump() for x in dataset[:10].to_alpaca()])
d = Dataset.from_list(dataset[:1000].to_messages().model_dump()['messages_list'])

In [4]:
from enum import Enum


class ConversationFormat(str, Enum):
    alpaca = "alpaca"
    openai = "openai"
    
ConversationFormat.openai.value

'openai'

In [51]:
from helpers.utils import get_ts_filename

get_ts_filename(".json",False)
# ".json".rsplit(".", 1)

PosixPath('_20240223171258275185.json')

In [9]:
from helpers.model_messages import ModelMessages


formatted = ModelMessages("mistralai/Mistral-7B-v0.1").model_format(messages.to_list()[:10])

In [10]:
print(formatted[0])

<s>[INST] {"question":"What is the most recent ranking of countries by corruption perception index in 2023?"} [/INST] {"question":"What is the most recent ranking of countries by corruption perception index in 2023?","tasks":["SEARCH: Find the most recent Corruption Perception Index rankings for 2023"],"can_i_answer":false} </s>


In [4]:
from datasets import Dataset

llm1_alpaca_dst = Dataset.from_json("dataset/llm1_alpaca.jsonl")
llm2_alpaca_dst = Dataset.from_json("dataset/llm2_alpaca.jsonl")

In [6]:
llm1_dataset_new = dataset.get_llm(LLMType.LLM1)
llm2_dataset_new = dataset.get_llm(LLMType.LLM2)
llm1_dataset_new = LLMDataset(rows=[x for x in llm1_dataset_new if x.input and x.output])
llm2_dataset_new = LLMDataset(rows=[x for x in llm2_dataset_new if x.input and x.output])
llm1_dataset_new = llm1_dataset_new.fill_systems(list(set(llm1_alpaca_dst['system'])))
llm2_dataset_new = llm2_dataset_new.fill_systems(list(set(llm2_alpaca_dst['system'])))

In [21]:
llm1_dataset_old = LLMDataset(rows=[DatasetRow.model_validate({"llm": LLMType.LLM1.value}|x) for x in llm1_alpaca_dst])
llm2_dataset_old = LLMDataset(rows=[DatasetRow.model_validate({"llm": LLMType.LLM2.value}|x) for x in llm2_alpaca_dst])

In [46]:
llm1_dataset = llm1_dataset_old + llm1_dataset_new
llm2_dataset = llm2_dataset_old + llm2_dataset_new

In [9]:
import json


def convert_to_dict(val: str | dict | None) -> dict | None:
    if val is None:
        return None
    if isinstance(val, str):
        return json.loads(val)
    return val


def convert_to_str(val: str | dict | None) -> str | None:
    if val is None:
        return None
    if isinstance(val, dict):
        return json.dumps(val)
    return val


_llm1_dataset = []
for i, x in enumerate(llm1_dataset_new):
    x.input = convert_to_dict(x.input)
    x.output = convert_to_dict(x.output)
    try:
        if isinstance(x.input, dict) and isinstance(x.output, dict):
            if x.output.get("question") is None:
                x.output["question"] = x.input["question"]
        _llm1_dataset.append(LLM1DatasetRow(**x.model_dump()))
    except Exception as e:
        print(i, type(x.input), type(x.output), e)

8306 <class 'dict'> <class 'dict'> 2 validation errors for LLM1DatasetRow
output.tasks.0
  Input should be a valid string [type=string_type, input_value={'INSTRUCTION': 'Provide ...a comparable platform.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
output.tasks.1
  Input should be a valid string [type=string_type, input_value={'SEARCH': 'Find the offi...ading iTunes for free.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
13769 <class 'dict'> <class 'dict'> 1 validation error for LLM1DatasetRow
input.question
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
19003 <class 'dict'> <class 'dict'> 5 validation errors for LLM1DatasetRow
output.tasks.0
  Input should be a valid string [type=string_type, input_value={'task': "DEFINE: Determi...its meaning and origin"}

In [65]:
_llm2_dataset = []
d = {}
c = 0
for i, x in enumerate(llm2_dataset):
    x.input = convert_to_dict(x.input)
    x.output = convert_to_str(x.output)
    try:
        d.setdefault((type(x.input), type(x.output)), []).append((i, x))
        # if isinstance(x.input, dict) and isinstance(x.output, dict):
        #     if x.output.get('question') is None:
        #         x.output['question'] = x.input['question']
        if isinstance(x.output, str):
            x.output = x.output.replace("eschema", "schema")
        x.output = convert_to_dict(x.output)
        _llm2_dataset.append(LLM2DatasetRow(uid=x.uid, system=x.system, input=x.input, output=x.output))
    except Exception as e:
        c += 1
        print(i, type(x.input), type(x.output), e)
c

35507 <class 'dict'> <class 'dict'> 1 validation error for LLM2DatasetRow
output.steps.1.extract.0.schema.1.description
  Field required [type=missing, input_value={'name': 'date_range', 't...tart_date', 'end_date']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


1

In [69]:
# llm1_dataset[24676].output
len(_llm1_dataset), len(_llm2_dataset)

(39094, 37367)

In [70]:
llm1_dataset = LLMDataset(rows=_llm1_dataset)
llm2_dataset = LLMDataset(rows=_llm2_dataset)

In [93]:
llm1_dataset.to_messages().model_dump()['messages_list']

[{'messages': [{'role': 'system',
    'content': "Being an honest and smart assistant talented in breaking down questions into actionable items, you're charged with interpreting a JSON-formatted question. Your output must be a JSON object articulated with two keys: can_i_answer (indicating true if the inquiry is answerable using internal capabilities, or false if it requires external resources) and tasks, delineating the series of steps to answer the question with external aids if can_i_answer is false."},
   {'role': 'user',
    'content': '{"question":"What is the GDP growth rate of India in 2021?"}'},
   {'role': 'assistant',
    'content': '{"question":"What is the GDP growth rate of India in 2021?","tasks":["SEARCH,DATA: Retrieve the GDP growth rate data for India for the year 2021"],"can_i_answer":false}'}]},
 {'messages': [{'role': 'system',
    'content': "In your capacity as a sincere and intelligent assistant expert in breaking down complex queries into accessible steps, your

In [94]:
llm1_d = Dataset.from_list(llm1_dataset.to_messages().model_dump()['messages_list'])
llm2_d = Dataset.from_list(llm2_dataset.to_messages().model_dump()['messages_list'])

In [5]:
from json import loads
from helpers.text_utils import TextUtils

prompt = "<<SYS>> Being an honest and smart assistant talented in breaking down questions into actionable items, you're charged with interpreting a JSON-formatted question. Your output must be a JSON object articulated with two keys: can_i_answer (indicating true if the inquiry is answerable using internal capabilities, or false if it requires external resources) and tasks, delineating the series of steps to answer the question with external aids if can_i_answer is false. <<SYS>> [INST] {\"question\": \"What is the average salary in the technology industry in the United Kingdom?\"} [/INST] "

template = "<<SYS>> {system} <<SYS>> [INST] {input} [/INST] {output} </s>"

output = "<s> <<SYS>> Being an honest and smart assistant talented in breaking down questions into actionable items, you're charged with interpreting a JSON-formatted question. Your output must be a JSON object articulated with two keys: can_i_answer (indicating true if the inquiry is answerable using internal capabilities, or false if it requires external resources) and tasks, delineating the series of steps to answer the question with external aids if can_i_answer is false. <<SYS>> [INST] {\"question\": \"What is the average salary in the technology industry in the United Kingdom?\"} [/INST] {\"question\": \"What is the average salary in the technology industry in the United Kingdom?\", \"splits\": [\"SEARCH,DATA: Retrieve the average salary data for the technology industry in the United Kingdom\"], \"can_i_answer\": false} </s>"

TextUtils.get_middle_text(output, prompt, "</s>", loads)
TextUtils.parse_to_dict(template, output)

{'system': "Being an honest and smart assistant talented in breaking down questions into actionable items, you're charged with interpreting a JSON-formatted question. Your output must be a JSON object articulated with two keys: can_i_answer (indicating true if the inquiry is answerable using internal capabilities, or false if it requires external resources) and tasks, delineating the series of steps to answer the question with external aids if can_i_answer is false.",
 'input': '{"question": "What is the average salary in the technology industry in the United Kingdom?"}',
 'output': '{"question": "What is the average salary in the technology industry in the United Kingdom?", "splits": ["SEARCH,DATA: Retrieve the average salary data for the technology industry in the United Kingdom"], "can_i_answer": false}'}